In [0]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/LMBTrain

In [0]:
!pip install keras==2.0.6

In [0]:
!pip install tensorflow-gpu==1.5.0

In [0]:
from modelalgo import *
import keras
from keras.models import load_model
keras.losses.weighted_binary_crossentropy = weighted_binary_crossentropy

# Settings
input_dim = 72
output_dim = 2
nseqlen = 128

test_dir =  "data/csv/test/"
train_dir = "data//csv/train/"

In [0]:
# Load data
inputs, outputs, ids = load_data(train_dir, input_dim, output_dim, nseqlen, nsamples = 900)
inputs_test, outputs_test, ids_test = load_data(test_dir, input_dim, output_dim, nseqlen, nsamples = 900)

ntrain = inputs.shape[0]

inputs = np.concatenate((inputs, inputs_test), axis=0)
outputs = np.concatenate((outputs, outputs_test), axis=0)
ids = np.concatenate((ids, ids_test), axis=0)

n = 12

In [0]:
# Combinations
hiddens = [128]
lstmlays = [3]
products = list(itertools.product(hiddens, lstmlays))

# Run
results = {}
tt = time.strftime('%Y%m%d%H%M%s')

#Seeing Lukas's weight
# model = load_model("models/FO.h5")
#model = load_model("models/HS.h5")
#model = load_model("model-2020022111071582283276-128-3.h5")

In [0]:
for exam in products:
    # needs to try a few combinations
    # cols = range(72) # everything
    # cols = range(12) + [21 + i for i in range(6)] + [28 + i for i in range(6)]
    cols = [2 + i for i in range(10)] + [24 + i for i in range(12)] + [57 + i for i in range(12)]
    model = construct_model(hidden=128, lstm_layers=3, input_dim=len(cols), output_dim=1)
    history = model.fit(inputs[0:ntrain, :, cols], outputs[0:ntrain, :, 1:2], nb_epoch=500, batch_size=32, verbose=2, validation_split=0.2)  #HS 
    # history = model.fit(inputs[0:ntrain, :, cols], outputs[0:ntrain, :, 0:1], nb_epoch=500, batch_size=32, verbose=2, validation_split=0.2) #FO
    plot_history(history)

    # Test on the test set
    # Needs to train seperately for HS and FO
    scores = model.evaluate(inputs[ntrain:n, :, cols], outputs[ntrain:n, :, 1:2]) #HS training
    # scores = model.evaluate(inputs[ntrain:n, :, cols], outputs[ntrain:n, :, 0:1]) #FO training
    print("Test set loss: %.4f" % (scores[0]))
    sdist = []
    res = model.predict(inputs[ntrain:n, :, cols])

    for ntrial in range(n - ntrain):
        likelihood = res[ntrial, :, 0:1]
        true = outputs[ntrain + ntrial, :, 1:2]
        d = eval_prediction(likelihood, true, ids[ntrain + ntrial], plot=False, shift=0)
        sdist.extend(d)
        if (d[0] > 10):
            eval_prediction(likelihood, true, ids[ntrain + ntrial], plot=True, shift=0)

    # plot_stats(sdist)
    model.save("model-%s-%d-%d.h5" % (tt, exam[0], exam[1]))
  

In [0]:
sdist = []
for Filename_In in os.listdir(test_dir):
  if not Filename_In.startswith('.') and os.path.isfile(os.path.join(test_dir, Filename_In)):
      R = np.loadtxt("%s/%s" % (test_dir, Filename_In), delimiter=',')
      X = R[:, cols]
      true = R[:, (input_dim + 1):(input_dim + 2)]
      likelihoodHS = model.predict(X.reshape((1, -1, len(cols))))[0]
            # likelihoodFO = modelFO.predict(X.reshape((1, -1, len(cols))))[0]
            # sdistFO.extend(eval_prediction(likelihoodFO, true, filename))
      sdist.extend(eval_prediction(likelihood, true, Filename_In))